In [1]:
from bs4 import BeautifulSoup as beauty
import cloudscraper
from urllib.parse import urlparse
from urllib.parse import parse_qs

In [7]:
from os.path import exists
import time


scraper = cloudscraper.create_scraper(delay=15, browser='chrome') 

def do_the_magic(i):  
    
    # print("start - processing " + str(i))

    file_path = "/home/alvaro/scrap/" + str(i) + ".html"
    if exists(file_path):
        # print("skiping - file already exists for " + str(i))
        return

    retry_count = 0

    time.sleep(1)
    while(True):
        url_base = 'https://www.tibia.com/charactertrade/?subtopic=pastcharactertrades&page=details&auctionid=' 
        url = url_base + str(i)
        info = scraper.get(url).text

        if "An internal error has occurred. Please try again later!" in info:
            #print("skiping - id not found for " + str(i))
            break


        if "403 Forbidden" in info:
            # print("retry - forbiden for " + str(i))
            if retry_count <= 4:
                retry_count += 1
                time.sleep(2)
                continue
            else:
                # print('retry - count exceeded for ' + str(i))
                retry_count = 0
                f = open('/home/alvaro/failed5.txt', "a")
                f.write(str(i) + '\n')
                f.close()
                break

        retry_count = 0
        # print('success - saving for ' + str(i))
        f = open(file_path, "w+")
        f.write(info)
        f.close()
        break
    

In [3]:
from multiprocessing import Pool
import tqdm
import os

last_id=970000

pool = Pool(3)

results = [x for x in tqdm.tqdm(pool.imap_unordered(
        do_the_magic,reversed(range(954390, int(last_id)))), total=last_id-954390)]

pool.close()
pool.join()
   

100%|███████████████████████████████████| 15610/15610 [1:14:19<00:00,  3.50it/s]


In [8]:

with open('/home/alvaro/failed4.txt') as f:
    lines = [line.strip() for line in f]

lines_int = []

for i in range(len(lines)):
    try:
        n = int(lines[i])
        lines_int.append(n)
    except:
        print(i)

        
from multiprocessing import Pool
import tqdm
import os

pool = Pool(3)

    
results = [x for x in tqdm.tqdm(pool.imap_unordered(
        do_the_magic, lines_int), total=len(lines_int))]

pool.close()
pool.join()

FileNotFoundError: [Errno 2] No such file or directory: '/home/alvaro/failed4.txt'